# Этап 0. Импорт библиотек и описание процесса решения задачи

In [45]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


__Задача:__ Заполнить как можно больше пустых значений в колонках *product_type* и *product_category* для тех товаров, где этих данных нет

Работа над решением проблемы должна проводиться по алгоритму:

Список действий:
1. Удалить из таблицы ненужные признаки - все product_type кроме en, ru и de.
2. Обработать комментарии на русском языке:
   1. Удалить название товара
   2. Удалить название компании
   3. Подчистить запись и перезаписать без знаков препинания в виде списка ключевых слов
   4. Найти описания, которые содержат название группы по уже существующим объектам
   5. Вписать название в категорию для нового объекта
3. Повторить действия для комментариев на английском и немецком языках
4. Обработать фотографии, по ключевым словам, взять ключевые слова из описания, как название категории и сравнить их с ключевыми словами, которые удалось получить по фотографии
5. В случае получения идентичных слов - заполнить категорию

# Этап 1. Подгрузка данных

Подгрузим данные и проанализируем их

In [47]:
data = pd.read_csv('/content/drive/MyDrive/Rumatica/Август/август2/product_random description_type_multi_lang.csv', sep=';')
data.head()

,id,brand_name,article,product_name,product_type_ru,product_type_en,product_type_de,product_type_ua,product_type_es,product_type_pt,...,product_type_ro,product_type_nl,product_type_bg,product_type_mk,product_category,photo,famaga_photo,description_ru,description_de,description_und
0,118048,Bernstein,6018200002,SLC-M-024-11/11-R4,Защитный выключатель блокировки,Guard locking safety switch,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,NaN,"SLC-M-024-11/11-R4, Safety switch: blocking; S..."
1,118053,Bernstein,6018200007,SLC-F-024-20/20-R4,Защитный выключатель блокировки,Guard locking safety switch,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,NaN,model SLC-F-024-20/20-R4
2,118054,Bernstein,6018200008,SLC-F-024-11/20-R4,Защитный выключатель блокировки,Guard locking safety switch,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,NaN,1. Safety interlock switch SLC-F-024-11/20-R4 ...
3,118074,Bernstein,6018119045,SLK-F-UC-55-R1-A0-L0-0,NaN,Switches with guard locking,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,NaN,переключатель \nпоследующая модель для 6018119...,"SLK-F-UC-55-R1-A0-L0-0 , Sicherheitschalter [ ...",BERNSTEIN GERMANY limit sw slk-FVTU24UC55ARX H...
4,118076,Bernstein,6018119047,SLK-M-UC-55-R0-A0-L0-0,NaN,Switches with guard locking,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,NaN,выключатель SLK-M-UC-55-R0-A0-L0-0 A,NaN,SLK-M-UC-55-R0-A0-L0-0 A


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356960 entries, 0 to 356959
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                356960 non-null  int64  
 1   brand_name        356960 non-null  object 
 2   article           350683 non-null  object 
 3   product_name      26466 non-null   object 
 4   product_type_ru   19840 non-null   object 
 5   product_type_en   17979 non-null   object 
 6   product_type_de   9928 non-null    object 
 7   product_type_ua   0 non-null       float64
 8   product_type_es   5481 non-null    object 
 9   product_type_pt   0 non-null       float64
 10  product_type_it   2288 non-null    object 
 11  product_type_fr   5389 non-null    object 
 12  product_type_pl   2267 non-null    object 
 13  product_type_cs   2188 non-null    object 
 14  product_type_hu   2152 non-null    object 
 15  product_type_ro   6317 non-null    object 
 16  product_type_nl   0 

Перед нами находятся данные, содержащие 356960 строк и 25 столбцов.

Всего из 356960 значений в столбце:

__product\_category__ заполнено __10158__ значений,

в __product\_type\_ru__ - __19840 значений__,

в  __product\_type\_en__ - __17979 значений__,

а в __product\_type\_de__ - __9928 значений__.

Остальные столбцы описаний либо не содержат данных, либо содержат их в слишком мелком для работы количестве.

Очистим данные от неинформативных столбцов, которые не получится использовать для получения данных о категориях и типах товаров


In [49]:
data = data.drop(['id',
 'article',
 'product_type_ua',
 'product_type_es',
 'product_type_pt',
 'product_type_it',
 'product_type_fr',
 'product_type_pl',
 'product_type_cs',
 'product_type_hu',
 'product_type_ro',
 'product_type_nl',
 'product_type_bg',
 'product_type_mk',
 'famaga_photo'], axis=1)

In [50]:
data.head()

,brand_name,product_name,product_type_ru,product_type_en,product_type_de,product_category,photo,description_ru,description_de,description_und
0,Bernstein,SLC-M-024-11/11-R4,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,"SLC-M-024-11/11-R4, Safety switch: blocking; S..."
1,Bernstein,SLC-F-024-20/20-R4,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,model SLC-F-024-20/20-R4
2,Bernstein,SLC-F-024-11/20-R4,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,1. Safety interlock switch SLC-F-024-11/20-R4 ...
3,Bernstein,SLK-F-UC-55-R1-A0-L0-0,NaN,Switches with guard locking,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,переключатель \nпоследующая модель для 6018119...,"SLK-F-UC-55-R1-A0-L0-0 , Sicherheitschalter [ ...",BERNSTEIN GERMANY limit sw slk-FVTU24UC55ARX H...
4,Bernstein,SLK-M-UC-55-R0-A0-L0-0,NaN,Switches with guard locking,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,выключатель SLK-M-UC-55-R0-A0-L0-0 A,NaN,SLK-M-UC-55-R0-A0-L0-0 A


Заполним пропуски в столбцах с описанием на прочерк, а также заранее переведем все данных в столбцах описаний в строковый формат данных

In [51]:
for col in ['description_ru',	'description_de',	'description_und']:
  data[col] = data[col].fillna('-')
  data[col] = data[col].apply(lambda x: str(x))

Удалим из всех столбцов описаний название продутка и бренда, предварительно переведя как названия, так и данные в столбцах в нижний регистр

In [52]:
for col in ['description_ru',	'description_de',	'description_und']:
  data[col] = data.apply(lambda row: row[col].upper().replace(str(row['product_name']).upper(), ''), axis=1)
  data[col] = data.apply(lambda row: row[col].lower().replace(str(row['brand_name']).lower(), ''), axis=1)

Проверяем, что в данных описаний не сохранилось информации о названии продутка и компании

In [53]:
for col in ['description_ru',	'description_de',	'description_und']:
  data[col+'_have_product_name'] = '-'
  data[col+'_have_brand_name'] = '-'

In [59]:
for col in ['description_ru',	'description_de',	'description_und']:
  for i in tqdm(range(data.shape[0])):
    data[col+'_have_product_name'].iloc[i] = True if str(data['product_name'].iloc[i]).lower() in data[col].iloc[i] else False
    data[col+'_have_brand_name'].iloc[i] = True if str(data['brand_name'].iloc[i]).lower() in data[col].iloc[i] else False

 17%|█▋        | 61270/356960 [00:50<05:15, 936.03it/s]Exception ignored in: <function tqdm.__del__ at 0x78978eea1bd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1277, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'
100%|██████████| 356960/356960 [04:50<00:00, 1227.99it/s]


Выведем информацию о вхождениях по столбцам. Если есть True - значит в какой-то из записей осталась информация о компании, или о названии продукта. Если все выборки выдают False yа 356960 значений, то значит что данные были почищены корректно.

In [61]:
for col in ['description_ru',	'description_de',	'description_und']:
  display(data[col+'_have_product_name'].value_counts())
  display(data[col+'_have_brand_name'].value_counts())

,count
description_ru_have_product_name,
False,356960


,count
description_ru_have_brand_name,
False,356960


,count
description_de_have_product_name,
False,356960


,count
description_de_have_brand_name,
False,356960


,count
description_und_have_product_name,
False,356960


,count
description_und_have_brand_name,
False,356960


Удалим неинформативные столбцы из данных - столбцы, которые были созданы для проверки корректности очистки данных, а также данные о названии компании и продукта

In [63]:
data2 = data.copy()

In [67]:
data = data.drop(['brand_name',
 'product_name',
 'description_ru_have_product_name',
 'description_ru_have_brand_name',
 'description_de_have_product_name',
 'description_de_have_brand_name',
 'description_und_have_product_name',
 'description_und_have_brand_name'], axis=1)

In [68]:
data.head()

,product_type_ru,product_type_en,product_type_de,product_category,photo,description_ru,description_de,description_und
0,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,", safety switch: blocking; slc; contacts: nc+n..."
1,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,model
2,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,1. safety interlock switch (6018200008) - 1 pc.
3,NaN,Switches with guard locking,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,переключатель \nпоследующая модель для 6018119...,", sicherheitschalter [ nachfolger von 6018119...","germany limit sw slk-fvtu24uc55arx ho,sicheru..."
4,NaN,Switches with guard locking,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,выключатель a,-,a


Сохраним файл для более простой дальнейшей работы

In [69]:
data.to_csv('Подготовленные для заполнения типов и категорий данные.csv', sep=';')

# Этап 2. Обработка отзывов и подготовка данных для поиска типов и категорий